# CIFAR10 in Swift for TensorFlow (ConvNet)

## Importing dependencies

In [0]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))' Datasets

In [0]:
import TensorFlow
import Foundation
import Datasets

In [0]:
import Python
let plt = Python.import("matplotlib.pylab")
let np = Python.import("numpy")

In [0]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

## Loading CIFAR10

[tensorflow/swift-models](https://github.com/tensorflow/swift-models)

In [0]:
let batchSize = 512

In [0]:
var dataset = CIFAR10()

## Inspect examples

In [0]:
var iterator = dataset.trainingDataset.makeIterator()
let image = iterator.next()!.data.reshaped(to: TensorShape(32, 32, 3)).makeNumpyArray()
plt.imshow(image)
plt.show()

In [0]:
let image = iterator.next()!.data.reshaped(to: TensorShape(32, 32, 3)).makeNumpyArray()
plt.imshow(image)
plt.show()

## Constructing the network

In [0]:
struct Model: Layer {
    var flatten = Flatten<Float>()
    var dropout25 = Dropout<Float>(probability: 0.25)

    // 1
    var conv1 = Conv2D<Float>(
      filterShape: (3, 3, 3, 32),
      padding: .same,
      activation: relu
    )
    var conv2 = Conv2D<Float>(
      filterShape: (3, 3, 32, 32),
      padding: .same,
      activation: relu
    )
    var maxPooling1 = MaxPool2D<Float>(poolSize: (2, 2), strides: (1, 1))
    // dropout

    // 2
    var conv3 = Conv2D<Float>(
      filterShape: (3, 3, 32, 64),
      padding: .same,
      activation: relu
    )
    var conv4 = Conv2D<Float>(
      filterShape: (3, 3, 64, 64),
      padding: .same,
      activation: relu
    )
    var maxPooling2 = MaxPool2D<Float>(poolSize: (2, 2), strides: (1, 1))
    // dropout

    var dense1 = Dense<Float>(inputSize: 30 * 30 * 64, outputSize: 128, activation: relu)
    var dropout50 = Dropout<Float>(probability: 0.5)
    var dense2 = Dense<Float>(inputSize: 128, outputSize: 10, activation: softmax)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input
          .sequenced(through: conv1, conv2, maxPooling1, dropout25)
          .sequenced(through: conv3, conv4, maxPooling2, dropout25)
          .sequenced(through: flatten, dense1, dropout50, dense2)
    }
}

In [0]:
var model = Model()

In [0]:
let dummy = Tensor<Float>(randomNormal: TensorShape(1, 32, 32, 3))
model(dummy)

## Training

In [0]:
let epochs = 50
var trainHistory = np.zeros(epochs)
var valHistory = np.zeros(epochs)

In [0]:
let optimizer = Adam(for: model)

In [0]:
for epoch in 0..<epochs {
    // Update parameters
    Context.local.learningPhase = .training
    let trainingShuffled = dataset.trainingDataset.shuffled(sampleCount: dataset.trainingExampleCount, randomSeed: Int64(epoch)) 
    for batch in trainingShuffled.batched(batchSize) {
        let (labels, images) = (batch.label, batch.data)
        let (_, gradients) = valueWithGradient(at: model) { model -> Tensor<Float> in
            let logits = model(images)
            return softmaxCrossEntropy(logits: logits, labels: labels)
        }
        optimizer.update(&model, along: gradients)
    }

    // Evaluate model
    Context.local.learningPhase = .inference

    var correctTrainGuessCount = 0
    var totalTrainGuessCount = 0
    dataset.trainingDataset.batched(512).forEach { batch in
        let images = batch.data, labels = batch.label
        let logits = model(images)
        let correctPredictions = logits.argmax(squeezingAxis: 1) .== labels
        correctTrainGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
    }
    let trainAcc = Float(correctTrainGuessCount) / Float(dataset.trainingExampleCount)
    trainHistory[epoch] = PythonObject(trainAcc)

    var correctValGuessCount = 0
    var totalValGuessCount = 0
    dataset.testDataset.batched(512).forEach { batch in
        let images = batch.data, labels = batch.label
        let logits = model(images)
        let correctPredictions = logits.argmax(squeezingAxis: 1) .== labels
        correctValGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
    }
    let valAcc = Float(correctValGuessCount) / Float(10000)
    valHistory[epoch] = PythonObject(valAcc)
    
    print("\(epoch + 1) | Training accuracy: \(trainAcc) | Validation accuracy: \(valAcc)")
}

## Inspecting training history

In [0]:
plt.plot(trainHistory)
plt.title("Training History")
plt.show()

In [0]:
plt.plot(valHistory)
plt.title("Validation History")
plt.show()

## Making predictions

In [0]:
var iterator = dataset.testDataset.makeIterator()
let example = iterator.next()!
let image = example.data.reshaped(to: TensorShape(1, 32, 32, 3)) // batch size 1
print(model(image).argmax(), example.label)

In [0]:
let example = iterator.next()!
let image = example.data.reshaped(to: TensorShape(1, 32, 32, 3)) // batch size 1
print(model(image).argmax(), example.label)

In [0]:
let example = iterator.next()!
let image = example.data.reshaped(to: TensorShape(1, 32, 32, 3)) // batch size 1
print(model(image).argmax(), example.label)

In [0]:
let example = iterator.next()!
let image = example.data.reshaped(to: TensorShape(1, 32, 32, 3)) // batch size 1
print(model(image).argmax(), example.label)